# kNN으로 타이타닉 탑승객 생존률 예측

## 1. 데이터 준비

In [1]:
import pandas as pd
import seaborn as sns

# Seaborn에 내장된 타이타닉 자료를 호출
df = sns.load_dataset('titanic')

# data 확인
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True



- survivied: 생존여부 (0: 사망, 1: 생존)
- pclass: 좌석 등급 (1등급, 2등급, 3등급)
- sex: 성별
- age: 나이
- sibsp: 같이 탑승한 형제, 자매 또는 배우자 인원 수
- parch: 같이 탑승한 부모님 또는 자녀 인원 수 (파치)
- fare: 좌석 요금 (페어)
- embarked: 탑승 항구 (C = Cherbourg, Q = Queenstown, S = Southampton) 임바크
- 셰르부르옥트빌(프랑스북부), 퀸스타운(뉴질랜드 남섬의 도시), 사우샘프턴(영국 남부)
- class: pclass와 동일
- who: 성별과 동일
- adult_male: 성인 남자 여부
- deck: 데크 번호 (알파벳 + 숫자 혼용)
- embark_town: 탑승 항구 이름
- alive: 생존여부 (yes, no)
- alone: 혼자 탑승 여부


## 2. 데이터 탐색

데이터의 특성과 분석 목표에 맞춰 누락 데이터를 처리한다.

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


#### 결측 데이터(deck)와 중복 데이터(embark_town)를 처리함 (나이는 결측치가 많지만, 생존여부에 중요한 변수가 될 수 있음)

In [3]:
rdf = df.drop(['deck','embark_town'],axis=1)
rdf.columns.values

array(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'alive', 'alone'],
      dtype=object)

#### age 필드 작업 : dropna()

행이나 열에 값이 없는 부분을 제외시킨다.

In [4]:
rdf = rdf.dropna(subset=['age'], how='any', axis=0)
print(len(rdf))

714


#### idxmin(), idxmax()

행 또는 열에서 가장 작은 값과 / 큰 값을 반환한다.

- 'embarked' 필드에서 결측치 2개 가장 빈도가 높은 값으로 치환

In [20]:
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()
print(most_freq)

print(rdf.describe(include='all'))

rdf['embarked'].fillna(most_freq, inplace=True)

S
          survived      pclass   sex         age       sibsp       parch  \
count   714.000000  714.000000   714  714.000000  714.000000  714.000000   
unique         NaN         NaN     2         NaN         NaN         NaN   
top            NaN         NaN  male         NaN         NaN         NaN   
freq           NaN         NaN   453         NaN         NaN         NaN   
mean      0.406162    2.236695   NaN   29.699118    0.512605    0.431373   
std       0.491460    0.838250   NaN   14.526497    0.929783    0.853289   
min       0.000000    1.000000   NaN    0.420000    0.000000    0.000000   
25%       0.000000    1.000000   NaN   20.125000    0.000000    0.000000   
50%       0.000000    2.000000   NaN   28.000000    0.000000    0.000000   
75%       1.000000    3.000000   NaN   38.000000    1.000000    1.000000   
max       1.000000    3.000000   NaN   80.000000    5.000000    6.000000   

              fare embarked  class  who adult_male alive alone  
count   714.000000  

In [17]:
# ** 참고 **
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

df = DataFrame([[1.4, np.nan],[7.10, -4.5],[np.nan, np.nan],[0.75, -1.3]],
              index = ['A','B','C','D'],
              columns = ['ONE','TWO'])
print(df)
print('-'*20)
print(df.idxmin())
print('-'*20)
print(df.idxmax())

    ONE  TWO
A  1.40  NaN
B  7.10 -4.5
C   NaN  NaN
D  0.75 -1.3
--------------------
ONE    D
TWO    B
dtype: object
--------------------
ONE    B
TWO    D
dtype: object


## 3. 분석에 사용할 속성 지정

In [27]:
ndf = rdf[['survived','pclass','sex','age','sibsp','parch','embarked']]
print(ndf.head())

   survived  pclass     sex   age  sibsp  parch embarked
0         0       3    male  22.0      1      0        S
1         1       1  female  38.0      1      0        C
2         1       3  female  26.0      0      0        S
3         1       1  female  35.0      1      0        S
4         0       3    male  35.0      0      0        S


### One-Hot-Encoding

- 원핫인코딩은 범주형 데이터를 숫자형으로 변환한다.

In [28]:
# 성별을 0과 1로 바꿔주는 더미변수 생성 후 concat 후 이전꺼 삭제
onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf, onehot_sex], axis=1)

# 'embarked'를 숫자로 바꿔주는 더미변수 생성 후 concat 후 이전꺼 삭제
onehot_embarked = pd.get_dummies(ndf['embarked'],prefix='town')
ndf = pd.concat([ndf, onehot_embarked], axis=1)

ndf.drop(['sex','embarked'],axis=1, inplace=True)
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


### 훈련용 데이터 / 테스트용 데이터 구분

In [30]:
# 학습용 데이터
X = ndf[['pclass','age','sibsp','parch','female','male','town_C','town_Q','town_S']]

# 테스트용 데이터
y = ndf['survived']

# 데이터 변수들을 정규화(0 ~ 1,0 수치값)
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

# 학습용 70%, 테스트용 30% 데이터 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 10)

print('train_data 개수 : ', X_train.shape)
print('test_data 개수 : ', X_test.shape)

train_data 개수 :  (499, 9)
test_data 개수 :  (215, 9)


# kNN 알고리즘으로 데이터 분석

In [33]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(X_train, y_train)

y_hat = knn.predict(X_test)

print(y_hat[0:15]) # 예측값
print()
print(y_test.values[0:15]) # 정답

[0 0 1 0 0 1 1 1 0 0 1 1 0 0 0]

[0 0 1 0 0 1 1 1 0 0 1 0 0 0 0]


## 모델 성능 평가

In [34]:
from sklearn import metrics

knn_matrix = metrics.confusion_matrix(y_test, y_hat)

knn_matrix 

array([[109,  16],
       [ 25,  65]], dtype=int64)

## 평가 지표

In [36]:
knn_report = metrics.classification_report(y_test, y_hat)
print(knn_report)

              precision    recall  f1-score   support

           0       0.81      0.87      0.84       125
           1       0.80      0.72      0.76        90

    accuracy                           0.81       215
   macro avg       0.81      0.80      0.80       215
weighted avg       0.81      0.81      0.81       215



- precision : 정답을 맞춘 횟수
- recall : 맞춘 것을 정답과 비교했을 때 퍼센트값
- f1-score : 정답을 맞춘 것의 신뢰도
- accuracy : 전체 정확도